In [1]:
import requests
import json
import os
import pandas as pd
import numpy as np

In [2]:
te = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto2/temperature.csv')

In [3]:
te.columns

Index(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa',
       'Nahariyya', 'Jerusalem'],
      dtype='object')

In [4]:
te.shape

(45253, 37)

In [5]:
nulls = [te[i].isna().value_counts() for i in te.columns]

In [6]:
print(nulls[2])

False    45252
True         1
Name: Portland, dtype: int64


### eliminamos columnas inecesarias (solo usa y canada)

In [7]:
def keep_columns(x, y):
    """
    Toma como argumento la lista de las columnas que quiero conservar y el nombre del data frame.
    Recorre el indice de columnas del data frame correspondiente y elimina las
    columnas que no quiero conservar
    """
    relevant = x
    for i in y.columns:
        if i not in relevant:
            y.drop(columns=[i], inplace=True)

In [8]:
keep_columns(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston'], te)

### Agrupamos por fecha y mediana

In [9]:
def mediana_mensual(x):
    x['Date'] = x.datetime.apply(lambda y: y[:7])
    return x.groupby("Date").median()

In [10]:
tem = mediana_mensual(te)

### transformamos a celsius

In [11]:
def celsius_conv(row):
    for i in tem.index:
           row[i] = row[i] - 273.15
    return row
          

temc = tem.apply(lambda row : celsius_conv(row))

In [12]:
temc.head(3)

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston
Date,,,,,,,,,,,,,,,,,,,,,
2012-10,10.050,11.630,15.730,10.630,19.260,18.85,20.35,23.740,13.83,9.730,...,11.370,23.24,15.505086,26.25,11.2400,10.11,14.190,14.750,10.600,13.150
2012-11,7.165,8.405,13.510,7.900,15.500,15.52,14.32,17.705,7.45,7.190,...,4.010,14.86,8.470000,21.66,3.4900,3.55,5.420,5.695,1.255,4.335
2012-12,4.025,5.110,10.665,5.065,13.245,13.13,8.82,11.405,1.31,0.275,...,2.455,15.96,9.195000,22.19,3.0025,0.75,4.575,4.490,-3.135,1.935


### Repetimos proceso con datos de humedad

In [13]:
h = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto2/humidity.csv')

In [14]:
keep_columns(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston'], h)

In [15]:
def mediana_mensual(x):
    x['Date'] = x.datetime.apply(lambda y: y[:7])
    return x.groupby("Date").median()

In [16]:
hm = mediana_mensual(h)

### Obtengo los datos de la API

In [27]:
def get_carbon_data(url):
    url = f"{url}"
    headers = {
    'x-rapidapi-host': "daily-atmosphere-carbon-dioxide-concentration.p.rapidapi.com",
    'x-rapidapi-key': "6f91c81d8fmshc83b86b1b6c6c5cp108956jsnef221ca08c9a"
    }
    response = requests.request("GET", url, headers=headers)
    t = response.json()
    c = pd.DataFrame(t['co2'])
    return c

In [64]:
c = get_carbon_data('https://daily-atmosphere-carbon-dioxide-concentration.p.rapidapi.com/api/co2-api')

#### Creamos cd con todos los años para analizar dsp

In [ ]:
def cod_agrupado(x):
    return x.groupby(["year", "month"]).agg({"cycle": "median"})

In [ ]:
cd = cod_agrupado(c)

In [ ]:
cd.head()

In [67]:
c.tail()

,year,month,day,cycle,trend
3887,2021,8,23,411.45,415.31
3888,2021,8,24,411.45,415.31
3889,2021,8,25,411.45,415.32
3890,2021,8,26,411.45,415.33
3891,2021,8,27,411.46,415.33


#### Escalamos a cd para que tenga solo los años que nos interesan

In [68]:
dates = pd.period_range(start='2011-01-01', end='2021-08-27', freq='D')

In [69]:
c['datetime'] = [str(dates[i]) for i in range(len(dates))]

In [70]:
c.head(3)

,year,month,day,cycle,trend,datetime
0,2011,1,1,391.26,389.73,2011-01-01
1,2011,1,2,391.29,389.73,2011-01-02
2,2011,1,3,391.33,389.74,2011-01-03


In [71]:
keep_columns(['datetime', 'cycle', 'trend'], c)

In [72]:
c['Date'] = c.datetime.apply(lambda y: y[:7])

In [73]:
co2 = c.groupby("Date").agg({"cycle": "median"})

In [74]:
display(co2)

,cycle
Date,
2011-01,391.680
2011-02,391.730
2011-03,391.760
2011-04,392.240
2011-05,392.410
...,...
2021-04,416.415
2021-05,416.750
2021-06,415.940


In [ ]:
tecm['Carbon Levels'] = 

In [60]:
#
for i in co2.index:
    if i in ['2011', '2018', '2019', '2020', '2021']:
        co2.drop([i], axis=0, inplace=True)

In [62]:
co2.drop(['2011-01'], axis=0, inplace=True)

In [63]:
co2.head()

,cycle
Date,
2011-02,391.73
2011-03,391.76
2011-04,392.24
2011-05,392.41
2011-06,391.56
